In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import cv2
import os

In [ ]:
def load_video_data(video_path, frames=16, resize=(64, 64)):
    frames_list = []
    cap = cv2.VideoCapture(video_path)
    
    # Get total frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_indices = np.linspace(0, total_frames-1, frames, dtype=int)
    
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, resize)
            frames_list.append(frame)
    
    cap.release()
    return np.array(frames_list)


In [ ]:
def prepare_data(dataset_dir, categories, frames=16, resize=(64, 64)):
    X = []
    Y = []
    
    for label, category in enumerate(categories):
        category_dir = os.path.join(dataset_dir, category)
        
        for video_name in os.listdir(category_dir):
            video_path = os.path.join(category_dir, video_name)
            if video_name.endswith('.avi'):
                video_data = load_video_data(video_path, frames, resize)
                X.append(video_data)
                Y.append(label)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X, Y


In [ ]:
dataset_dir = '/path/to/UCF101'  # Update with your UCF101 dataset path
categories = ['category1', 'category2', ...]  # Replace with actual categories

X, Y = prepare_data(dataset_dir, categories)

# Split dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
# Normalize data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
def build_3d_cnn(input_shape, num_classes):
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(2, 2, 2)),
        
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        
        Conv3D(128, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = build_3d_cnn(X_train.shape[1:], len(categories))

history = model.fit(X_train, Y_train, epochs=10, batch_size=8, validation_data=(X_test, Y_test))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
def predict_video(model, video_path, frames=16, resize=(64, 64)):
    video_data = load_video_data(video_path, frames, resize)
    video_data = np.expand_dims(video_data, axis=0)  # Add batch dimension
    
    prediction = model.predict(video_data)
    predicted_class = np.argmax(prediction)
    return predicted_class